In [43]:
!pip install -q pandas openpyxl pdfplumber spacy fitz nltk pytesseract pillow


In [44]:
import pandas as pd
import pdfplumber
import spacy
import re
import concurrent.futures
from openai import OpenAI

In [45]:
PDF_PATH = "/content/Data Protection Law - Part 2.pdf"

In [46]:
import pdfplumber
import re

def extract_text_from_pdf(pdf_path):
    pages_data = []

    with pdfplumber.open(pdf_path) as pdf:
        total_pages = len(pdf.pages)

        for i in range(total_pages):
            page = pdf.pages[i]  # Access the page
            page_text = page.extract_text()
            if page_text:  # Ensure there's text in the page
                # Clean the text (remove unwanted characters)
                cleaned_text = re.sub(r'n/', '', page_text)  # Remove occurrences of 'n/'
                cleaned_text = re.sub(r'\n+', ' ', cleaned_text)  # Replace multiple newlines with a single space
                pages_data.append({"page_number": i + 1, "text": cleaned_text})  # Store as a dictionary

    return pages_data

# Example usage
pages = extract_text_from_pdf(PDF_PATH)

# Print out the cleaned text of each page
for page_data in pages:
    print(f"Page {page_data['page_number']}:\n{page_data['text']}\n")


Page 1:
Data Protection Law Mag. Boris Kandov, LL.M.

Page 2:
Lecture Overview Rights of Data Subject 1. Data Protection Roles 2.

Page 3:
Basic information GDPR Application ● Scope of Application - Material [Art. 2] and Territorial [Art. 3] Personal Data ● Any information relating to an identified or identifiable natural person (‘data subject’) [Art 4(1)] ● Special Categories of Personal Data [Art 9] Processing of ● Any operation or set of operations which is performed on personal data or on sets of personal data, whether or Personal Data not by automated means [Art 4(2)] ● Non-exhaustive list Principles of ● Lawfulness, fairness and transparency, Processing ● Purpose limitation, ● Data minimisation, [Art. 5] ● Accuracy ● Storage limitation ● Integrity and confidentiality, ● Accountability

Page 4:
Rights of the Data Subject

Page 5:
Rights of the Data Subject ● Art. 12 - 22 GDPR - Rights ● Art. 23 – Restrictions ● Data controllers are required to safeguard the exercise of data subjec

In [47]:
import pdfplumber
import pytesseract
import re

def extract_text_from_pdf(pdf_path):
    pages_data = []

    with pdfplumber.open(pdf_path) as pdf:
        total_pages = len(pdf.pages)

        for i in range(total_pages):
            page = pdf.pages[i]
            page_text = page.extract_text() or ""
            cleaned_text = re.sub(r'n/', '', page_text)
            cleaned_text = re.sub(r'\n+', ' ', cleaned_text)

            ocr_texts = []
            for img_idx, img_dict in enumerate(page.images):
                # Crop the image from the PDF page using coordinates
                try:
                    cropped = page.crop((img_dict['x0'], img_dict['top'], img_dict['x1'], img_dict['bottom']))
                    pil_image = cropped.to_image(resolution=300).original
                    ocr_result = pytesseract.image_to_string(pil_image)
                    # Optional: Filter out image OCR results that are mostly whitespace or boilerplate
                    if len(ocr_result.strip()) > 15 and not re.match(r'^\s*universitat\s*wien\s*$', ocr_result, re.I):
                        ocr_texts.append(ocr_result.strip())
                except Exception as e:
                    pass  # Ignore errors on image processing

            # Combine original text and image OCR results
            if ocr_texts:
                all_text = cleaned_text + "\n[OCR IMAGE TEXT]:\n" + "\n".join(ocr_texts)
            else:
                all_text = cleaned_text

            pages_data.append({"page_number": i + 1, "text": all_text.strip()})

    return pages_data

# Example usage
pages = extract_text_from_pdf(PDF_PATH)

concatenated_text = ""
for page_data in pages:
    page_text = f"Page {page_data['page_number']}:\n{page_data['text']}\n"
    print(page_text)
    concatenated_text += page_text

print(concatenated_text)


Page 1:
Data Protection Law Mag. Boris Kandov, LL.M.
[OCR IMAGE TEXT]:
4 Universitat
wien

Page 2:
Lecture Overview Rights of Data Subject 1. Data Protection Roles 2.
[OCR IMAGE TEXT]:
4 Universitat
wien

Page 3:
Basic information GDPR Application ● Scope of Application - Material [Art. 2] and Territorial [Art. 3] Personal Data ● Any information relating to an identified or identifiable natural person (‘data subject’) [Art 4(1)] ● Special Categories of Personal Data [Art 9] Processing of ● Any operation or set of operations which is performed on personal data or on sets of personal data, whether or Personal Data not by automated means [Art 4(2)] ● Non-exhaustive list Principles of ● Lawfulness, fairness and transparency, Processing ● Purpose limitation, ● Data minimisation, [Art. 5] ● Accuracy ● Storage limitation ● Integrity and confidentiality, ● Accountability
[OCR IMAGE TEXT]:
4 Universitat
wien

Page 4:
Rights of the Data Subject
[OCR IMAGE TEXT]:
4 Universitat
wien

Page 5:
Right

## Chunking

In [48]:
concatenated_text

"Page 1:\nData Protection Law Mag. Boris Kandov, LL.M.\n[OCR IMAGE TEXT]:\n4 Universitat\nwien\nPage 2:\nLecture Overview Rights of Data Subject 1. Data Protection Roles 2.\n[OCR IMAGE TEXT]:\n4 Universitat\nwien\nPage 3:\nBasic information GDPR Application ● Scope of Application - Material [Art. 2] and Territorial [Art. 3] Personal Data ● Any information relating to an identified or identifiable natural person (‘data subject’) [Art 4(1)] ● Special Categories of Personal Data [Art 9] Processing of ● Any operation or set of operations which is performed on personal data or on sets of personal data, whether or Personal Data not by automated means [Art 4(2)] ● Non-exhaustive list Principles of ● Lawfulness, fairness and transparency, Processing ● Purpose limitation, ● Data minimisation, [Art. 5] ● Accuracy ● Storage limitation ● Integrity and confidentiality, ● Accountability\n[OCR IMAGE TEXT]:\n4 Universitat\nwien\nPage 4:\nRights of the Data Subject\n[OCR IMAGE TEXT]:\n4 Universitat\nwi

### Idea of a format

In [49]:
from google.colab import userdata
API_KEY = userdata.get('api_key')
client = OpenAI(api_key=API_KEY)

In [50]:
[
  {
    "question": "Which of the following rights, listed under Articles 12–22 of the GDPR, allows a data subject to receive their personal data in a structured, commonly used, machine-readable format and transmit it to another controller?",
    "answers": [
      "Right to object to processing",
      "Right to erasure (“right to be forgotten”)",
      "Right to data portability",
      "Right to restrict processing"
    ],
    "correct_answer": 2
  }
]


[{'question': 'Which of the following rights, listed under Articles 12–22 of the GDPR, allows a data subject to receive their personal data in a structured, commonly used, machine-readable format and transmit it to another controller?',
  'answers': ['Right to object to processing',
   'Right to erasure (“right to be forgotten”)',
   'Right to data portability',
   'Right to restrict processing'],
  'correct_answer': 2}]

In [51]:
def generate_prompt(text):
    prompt = f"""Split the following text into thematic sections. For each section, provide a short title and the corresponding text and a topic tag. Each section should cover a different concept that could be the basis for a quiz question.

**Output format**: a single JSON object with one key, "sections", whose value is an array of objects.
Each object must have three string fields:
  - "title": a short (3–5 word) label for the concept
  - "text": the exact excerpt of the source text for that concept
  - "topic": thematic topic of the question

**IMPORTANT:** Output the JSON as a single line, **without any line breaks or `\\n` escape characters** in the values or the structure. Everything must be on one line.

text:
{text}
"""
    return prompt


In [52]:
def clean_json_string(json_string):
    """
    Removes markdown-style code block wrappers like ```json ... ``` or ``` ... ```
    and returns the raw JSON string.
    """
    pattern = r'^\s*```(?:json)?\s*\n?(.*?)\s*```\s*$'
    match = re.match(pattern, json_string, re.DOTALL | re.IGNORECASE)
    if match:
        return match.group(1).strip()
    return json_string.strip()

In [53]:
def get_data_from_gpt(text):
    prompt = generate_prompt(text)

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a helpful assistant, that need to split a text into thematic chunks."},
            {"role": "user", "content": prompt}
        ],
        n=1,
        temperature=0.8,
    )

    message = response.choices[0].message.content
    cleaned_message = clean_json_string(message)
    cleaned_message = cleaned_message.replace('\\n', ' ').replace('\n', ' ')


    return cleaned_message

In [54]:

text = concatenated_text
responce = get_data_from_gpt(text)

In [55]:
responce

'{"sections":[{"title":"GDPR Scope of Application","text":"Basic information GDPR Application ● Scope of Application - Material [Art. 2] and Territorial [Art. 3] Personal Data ● Any information relating to an identified or identifiable natural person (‘data subject’) [Art 4(1)] ● Special Categories of Personal Data [Art 9] Processing of ● Any operation or set of operations which is performed on personal data or on sets of personal data, whether or Personal Data not by automated means [Art 4(2)] ● Non-exhaustive list Principles of ● Lawfulness, fairness and transparency, Processing ● Purpose limitation, ● Data minimisation, [Art. 5] ● Accuracy ● Storage limitation ● Integrity and confidentiality, ● Accountability","topic":"Data Protection Law"},{"title":"Rights of Data Subjects","text":"Rights of the Data Subject ● Art. 12 - 22 GDPR - Rights ● Art. 23 – Restrictions ● Data controllers are required to safeguard the exercise of data subject‘s rights ● Article 89(2) and (3) GDPR","topic":"

In [56]:
# If responce is a string, do this first:
import json
data = json.loads(responce)

In [57]:
for section in data["sections"]:
    print("Title:", section["title"])
    print("Text:", section["text"])
    print("Topic:", section["topic"])
    print("-" * 40)  # Separator line


Title: GDPR Scope of Application
Text: Basic information GDPR Application ● Scope of Application - Material [Art. 2] and Territorial [Art. 3] Personal Data ● Any information relating to an identified or identifiable natural person (‘data subject’) [Art 4(1)] ● Special Categories of Personal Data [Art 9] Processing of ● Any operation or set of operations which is performed on personal data or on sets of personal data, whether or Personal Data not by automated means [Art 4(2)] ● Non-exhaustive list Principles of ● Lawfulness, fairness and transparency, Processing ● Purpose limitation, ● Data minimisation, [Art. 5] ● Accuracy ● Storage limitation ● Integrity and confidentiality, ● Accountability
Topic: Data Protection Law
----------------------------------------
Title: Rights of Data Subjects
Text: Rights of the Data Subject ● Art. 12 - 22 GDPR - Rights ● Art. 23 – Restrictions ● Data controllers are required to safeguard the exercise of data subject‘s rights ● Article 89(2) and (3) GDPR


In [58]:
def generate_question(text):
    prompt = f"""You are an expert quiz generator.

Based on the provided text, generate exactly one quiz question with four answer options and only one correct answer.

**Important formatting instructions:**
- Output a valid JSON object (not an array) in a single line. Do not add line breaks or '\\n' escape characters anywhere.
- Use this structure:
{{
  "question": "...",
  "answers": ["...", "...", "...", "..."],
  "correct_answer": X
}}
- "question": should be a clear, fact-based multiple-choice question based on the text.
- "answers": should be a list of four options; only one is correct, others should be plausible but incorrect.
- "correct_answer": should be the **index** (0, 1, 2, or 3) of the correct answer in the list.
- **Do not** add explanations or wrap the output in code or markdown.

Text to use for the question:
{text}
"""
    return prompt


In [59]:
def get_data_from_gpt(text):
    prompt = generate_question(text)

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a helpful assistant, that need to generate quiz questions and answers."},
            {"role": "user", "content": prompt}
        ],
        n=1,
        temperature=0.8,
    )

    message = response.choices[0].message.content
    cleaned_message = clean_json_string(message)
    cleaned_message = cleaned_message.replace('\\n', ' ').replace('\n', ' ')

    parsed_response = json.loads(cleaned_message)

    return parsed_response

In [60]:
QUIZ = []

for element in data["sections"]:
    quiz_responce = get_data_from_gpt(element["text"])
    QUIZ.append(quiz_responce)


In [61]:
QUIZ


[{'question': 'What does GDPR stand for?',
  'answers': ['General Data Protection Regulation',
   'Global Data Privacy Regulation',
   'General Data Privacy Regulation',
   'Global Data Protection Regulation'],
  'correct_answer': 0},
 {'question': 'What do Articles 12 to 22 of the GDPR primarily focus on?',
  'answers': ['Data protection officer responsibilities',
   'Rights of the Data Subject',
   'Penalties for non-compliance',
   'Data controller obligations'],
  'correct_answer': 1},
 {'question': 'What is a requirement for how to communicate with data subjects under Art. 12 GDPR?',
  'answers': ['Using complex legal jargon',
   'Communicating in a concise and transparent manner',
   'Providing information only upon request',
   'Making the information difficult to access'],
  'correct_answer': 1},
 {'question': 'What does the right of access under Article 15 of the GDPR allow an individual to do?',
  'answers': ['Request the deletion of personal data',
   'Obtain confirmation of